In [ ]:
# RPA Project 1: Ethical Email Automation Bot

# Step 1: Gmail API Authentication and Email Fetching

# Requirements:
# - Google Cloud Project with Gmail API enabled
# - OAuth 2.0 credentials (token.json will be saved locally)

# Install necessary packages (run once)
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib --quiet

import os.path
import base64
import json
import re
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.text import MIMEText
import time

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

# Step 1A: Authenticate and build the Gmail service

def authenticate_gmail():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    service = build('gmail', 'v1', credentials=creds)
    return service

# Step 1B: Fetch Unread Emails (basic metadata)

def fetch_unread_emails(service, max_results=5):
    results = service.users().messages().list(userId='me', labelIds=['INBOX', 'UNREAD'], maxResults=max_results).execute()
    messages = results.get('messages', [])
    email_list = []

    for msg in messages:
        msg_data = service.users().messages().get(userId='me', id=msg['id']).execute()
        payload = msg_data.get('payload', {})
        headers = payload.get('headers', [])

        email_info = {
            'id': msg['id'],
            'subject': next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)'),
            'from': next((h['value'] for h in headers if h['name'] == 'From'), 'Unknown'),
            'snippet': msg_data.get('snippet')
        }
        email_list.append(email_info)

    return email_list

# Step 2: Categorize Email Type and Apply Ethical Filters

CATEGORY_RULES = {
    'job': ["resume", "interview", "hiring", "job"],
    'inquiry': ["question", "inquiry", "help", "clarify"],
    'promotion': ["offer", "sale", "discount", "buy now"]
}

BLOCKED_DOMAINS = ["noreply@", "marketing", "promo", "newsletter"]

REPLY_TEMPLATE = {
    'job': "Thank you for your interest. We'll get back to you regarding the opportunity.",
    'inquiry': "Thank you for reaching out. We'll respond to your question shortly.",
    'general': "Thanks for your message. We will get back to you if needed."
}

REPLY_LIMIT = 1  # reply once per sender in this run

reply_log = []

# Send an email reply using Gmail API

def send_reply(service, to_email, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to_email
    message['subject'] = f"Re: {subject}"
    raw_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}
    service.users().messages().send(userId='me', body=raw_message).execute()

# Categorize and decide if we should reply

def categorize_and_filter(email):
    text = (email['subject'] + " " + email['snippet']).lower()
    sender = email['from'].lower()

    if any(block in sender for block in BLOCKED_DOMAINS):
        return "blocked"

    for category, keywords in CATEGORY_RULES.items():
        if any(kw in text for kw in keywords):
            return category

    return "general"

# Step 3: Run ethical auto-responder with logs

def ethical_auto_responder():
    print("Authenticating Gmail...")
    gmail_service = authenticate_gmail()
    print("Fetching unread emails...")
    emails = fetch_unread_emails(gmail_service)

    print(f"\nProcessing Emails ({len(emails)}):")
    responded_senders = set()

    for email in emails:
        category = categorize_and_filter(email)
        sender_email = email['from']

        if category == "blocked":
            print(f"[BLOCKED] From: {sender_email}, Subject: {email['subject']}")
            continue

        if sender_email in responded_senders:
            print(f"[SKIPPED - Already Responded] {sender_email}")
            continue

        if category in REPLY_TEMPLATE:
            send_reply(gmail_service, sender_email, email['subject'], REPLY_TEMPLATE[category])
            print(f"[REPLIED] To: {sender_email}, Category: {category}")
            reply_log.append({
                'from': sender_email,
                'subject': email['subject'],
                'category': category,
                'status': 'replied'
            })
            responded_senders.add(sender_email)
        else:
            print(f"[IGNORED] From: {sender_email}, No reply rule matched")

    # Save log to file
    with open("reply_log.json", "w") as f:
        json.dump(reply_log, f, indent=2)

if __name__ == '__main__':
    ethical_auto_responder()
